In [1]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import time

In [2]:
def apply_threshold_hsv(roi, hsv_color, threshold):
    """Apply color thresholding in HSV color space to isolate specific color ranges in the ROI."""
    lower_bound = np.array([max(0, hsv_color[0] - threshold), max(0, hsv_color[1] - threshold), max(0, hsv_color[2] - threshold)])
    upper_bound = np.array([min(180, hsv_color[0] + threshold), min(255, hsv_color[1] + threshold), min(255, hsv_color[2] + threshold)])
    mask = cv2.inRange(roi, lower_bound, upper_bound)
    return mask

def reference_frame(frame, mask_bound, hsv_color_1, hsv_color_2, threshold=15):
    x1, y1, x2, y2 = mask_bound
    roi = frame[y1:y2, x1:x2]
    roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)  # Convert ROI to HSV

    # Apply threshold in HSV color space
    mask_1 = apply_threshold_hsv(roi_hsv, hsv_color_1, threshold)
    roi[mask_1 != 0] = [255, 0, 0]  # Highlight in BGR space for visibility
    cluster_dict_1 = find_clusters(mask_1)

    mask_2 = apply_threshold_hsv(roi_hsv, hsv_color_2, threshold)
    roi[mask_2 != 0] = [0, 0, 255]  # Highlight in BGR space for visibility
    cluster_dict_2 = find_clusters(mask_2)

    cluster_dict_2 = filter_noise_clusters(cluster_dict_2, 100)

    # Convert clusters to np.array for potential performance improvements
    for cluster_idx in cluster_dict_1:
        cluster_dict_1[cluster_idx] = np.array(cluster_dict_1[cluster_idx])

    for cluster_idx in cluster_dict_2:
        cluster_dict_2[cluster_idx] = np.array(cluster_dict_2[cluster_idx])

    return roi, cluster_dict_1, cluster_dict_2

def filter_noise_clusters(cluster_dict, size_threshold):
    """
    Filters clusters based on a minimum size threshold.

    Parameters:
    - cluster_dict (dict): A dictionary where each key represents a cluster index,
      and the value is a list of points belonging to that cluster.
    - size_threshold (int): The minimum number of points a cluster must have to be included.

    Returns:
    - dict: A new dictionary containing only the clusters that meet the size threshold.
    """
    
    filtered_clusters = {}
    for key, points in cluster_dict.items():
        if len(points) > size_threshold:
            filtered_clusters[key] = points
            
    return filtered_clusters

def find_clusters(mask):
    """Find clusters in the mask using DBSCAN."""
    
    y_coord, x_coord = np.where(mask != 0)
    if len(y_coord) == 0:
        return {}  # Return an empty dict if no points found
    
    coord_array = np.stack((y_coord, x_coord), axis=-1)
    sorted_array = coord_array[coord_array[:, 1].argsort()]
    dbscan = DBSCAN(eps=5, min_samples=10)
    clusters = dbscan.fit_predict(sorted_array)

    cluster_dict = {}
    for point, cluster_idx in zip(sorted_array, clusters):
        if cluster_idx != -1:
            cluster_dict.setdefault(cluster_idx, []).append(point.tolist())
            
    return cluster_dict


def highlight_clusters(frame_roi, cluster_dict, error_keys, color=(0, 0, 255)):
    """
    Highlights specified clusters in a region of interest by changing their pixel colors.

    Parameters:
    - frame_roi (numpy.ndarray): The region of interest from the frame where clusters are to be highlighted.
    - cluster_dict (dict): A dictionary containing clusters with their points. Each key in the dictionary
      represents a cluster index, and the value is a list of points (row, column pairs) belonging to that cluster.
    - error_keys (list): A list of keys indicating which clusters in the cluster_dict should be highlighted.
    - color (tuple): The BGR color value to use for highlighting. Default is red (0, 0, 255).

    Returns:
    - numpy.ndarray: The modified region of interest with specified clusters highlighted.
    """
    for key in error_keys:
        if key in cluster_dict:  # Check if the key exists in the cluster dictionary
            for row, col in cluster_dict[key]:
                frame_roi[row, col] = color
    return frame_roi
    

white_keys = ['C', 'D', 'E', 'F', 'G', 'A', 'B']
black_keys = ['C#', 'D#', 'F#', 'G#', 'A#']

In [3]:
cap = cv2.VideoCapture(0)
HSV_color_1 = (33, 70, 180)
HSV_color_2 = (160, 120, 200)
mask_bound = (0, 265, 640, 452)
print("starting loop", cap)

while cap.isOpened():
    success, frame_img = cap.read()

    if not success:
        print("Ignoring empty camera frame.")
        break

    roi, cluster_dict_1, cluster_dict_2 = reference_frame(frame_img, mask_bound, HSV_color_1, HSV_color_2, threshold=40)

    # Highlight clusters for visual feedback
    error_keys_1 = list(cluster_dict_1.keys())  # Simulating detected errors, adjust as needed
    error_keys_2 = list(cluster_dict_2.keys())  # Simulating detected errors, adjust as needed
    frame_img = highlight_clusters(roi, cluster_dict_1, error_keys_1, color=(0, 255, 0))  # Green for one set
    frame_img = highlight_clusters(frame_img, cluster_dict_2, error_keys_2, color=(0, 0, 255))  # Red for another

    # Display the resulting frame
    cv2.imshow('frame', frame_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()

starting loop < cv2.VideoCapture 0x7fc3a09073b0>


: 

In [1]:
cap =  cv2.VideoCapture(0)

def pick_color(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        pixel = frame_img[y, x]
        
        # Convert the color of the selected pixel to HSV
        hsv = cv2.cvtColor(np.uint8([[pixel]]), cv2.COLOR_BGR2HSV)
        hsv_value = hsv[0][0]
        print("HSV Value at ({}, {}): {}".format(x, y, hsv_value))


if cap.isOpened():
    success, frame_img = cap.read()
    cv2.namedWindow('image')
    cv2.setMouseCallback('image', pick_color)

    # Display the image and wait for a key press
    cv2.imshow('image', frame_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print('no camera')

HSV Value at (942, 736): [161 132 181]
HSV Value at (1008, 779): [160 116 198]
HSV Value at (1048, 743): [160 114 186]
HSV Value at (866, 731): [160 128 180]
HSV Value at (788, 723): [161 128 193]
HSV Value at (753, 774): [160 118 188]
HSV Value at (724, 727): [ 34  74 176]
HSV Value at (642, 722): [ 33  74 178]
HSV Value at (828, 725): [ 31  75 170]
HSV Value at (898, 736): [ 34  74 172]
HSV Value at (971, 735): [ 34  70 178]
HSV Value at (834, 720): [ 32  73 171]
HSV Value at (899, 746): [ 31  65 169]


KeyboardInterrupt: 